In [ ]:
# | default_exp postdown.MDDoc

In [ ]:
# | exporti
import logging

from fastcore.basics import patch_to

In [ ]:
# | exporti
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [ ]:
#| export
class MDDoc:
    COMMENT_BEGIN_FLAG = "<__COMMENT__BEGIN__FLAG__>"

    def __init__(self):
        self.md_struct = list()

In [ ]:
#| exporti
@patch_to(MDDoc)
def line(self: MDDoc, content):
    self.md_struct.append(content + "\n")


@patch_to(MDDoc)
def br(self: MDDoc):
    self.line("")


@patch_to(MDDoc)
def block(self: MDDoc, content):
    self.line(content)
    self.br()


@patch_to(MDDoc)
def italic(self: MDDoc, content):
    self.block(f"*{content}*")


@patch_to(MDDoc)
def bold(self: MDDoc, content):
    self.block(f"**{content}**")


@patch_to(MDDoc)
def text(self: MDDoc, content):
    self.block(content)


@patch_to(MDDoc)
def hr(self: MDDoc):
    self.block('----------------')


@patch_to(MDDoc)
def title(self: MDDoc, content, level=1):
    self.block("#" * level + " " + content)


In [ ]:
#| exporti
@patch_to(MDDoc)
def table(self: MDDoc, columns_name, rows):
    self.line(f'|{" | ".join(columns_name)}|')
    self.line("---".join(list("|" * (len(columns_name) + 1))))
    
    if rows:
        for i in rows:
            self.line("|{0}|".format("|".join(i)))
    else:
        self.line("|" * (len(columns_name) + 1))
    self.br()


In [ ]:
#| exporti
@patch_to(MDDoc)
def code_block(self: MDDoc, code, language=""):
    self.line(f"```{language}")
    for i in code.split("\n"):
        self.line(i.replace("\n", ""))
    self.line("```")
    self.br()

In [ ]:
#| exporti
@patch_to(MDDoc)
def comment_begin(self: MDDoc, origin=None):
    if origin:
        logger.debug("comment_begin: %s", origin)
    else:
        logger.debug("comment_begin")

    self.line(self.COMMENT_BEGIN_FLAG)


@patch_to(MDDoc)
def comment_end(self: MDDoc, origin=None):
    if origin:
        logger.debug("comment_end: %s", origin)
    else:
        logger.debug("comment_end")

    i = -1
    while True:
        if self.md_struct[i].startswith(self.COMMENT_BEGIN_FLAG):
            self.md_struct[i] = "> \n"
            break
        self.md_struct[i] = "> " + self.md_struct[i]
        i -= 1
    self.br()


@patch_to(MDDoc)
def output(self: MDDoc):
    self.hr()
    self.block("Generated with [Postdown][PyPI].")
    # It is apprporiate that credit is given to the postman tool with a link to the original URL of Titor, but adding an author just makes it looks like Titor is the author of the API.
    # self.block("Author: [Titor](https://github.com/TitorX)")
    self.line("[PyPI]:    https://pypi.python.org/pypi/Postdown")
    return "".join(self.md_struct)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()